In [1]:
import pandas as pd
import numpy as np
import openpyxl
import pyarrow as pa
import pyarrow.parquet as pq
import glob
import os
from pandas.io.formats.excel import ExcelFormatter
from pandas.io import gbq
from openpyxl.workbook import Workbook
from shutil import copyfile
from pathlib import Path

In [2]:
old_path = "D:\\EcoBee\\Internship Final Steps\\Updated All Canada_2"
old_parquet_files = glob.glob(os.path.join(old_path, "*.parquet"))
new_path = "D:\\EcoBee\\Internship Final Steps\\Updated All Canada_3"
new_parquet_files = glob.glob(os.path.join(new_path, "*.parquet"))

In [3]:
for file in old_parquet_files:
    df = (pd.read_parquet(file))
    UniqueNames = df.Identifier.unique()
    DataFrameDict = {elem : pd.DataFrame() for elem in UniqueNames}
    for key in DataFrameDict.keys():
        DataFrameDict[key] = df[:][df.Identifier == key]
        df_1= DataFrameDict[key]
        filepath = new_path + '/' + key + '.parquet'
        df['date'] = df['date_time'].dt.date
        df['hour'] = df['date_time'].dt.hour
        #df['hours'] =df['hour']
        x1= df.columns.get_loc("SensorOcc000")
        X=df.iloc[:,x1:x1+10]
        df['number_of_sensors']= X.apply(lambda x: x.count(), axis=1)
        y= X.count(axis='columns')
        c=y.replace([1,2,3,4,5,6,7,8,9],1)
        df['number_of readings']= c
        X['date']= df['date']
        X['hour']= df['hour']
        #X['hours']=df['hours']
        dy= X.groupby(['date', 'hour']).mean()
        dz= pd.DataFrame()
        dz['date']= df['date']
        dz['hour']= df['hour']
        dz['no_sensors']= df['number_of_sensors']
        dz['number_of readings']= df['number_of readings']
        dg= dz.groupby(['date', 'hour']).max()
        dg1= dz.groupby(['date', 'hour']).sum()
        dy['no_of_sensors']= dg['no_sensors']
        dy['number_of readings']= dg1['number_of readings']
        x1= dy.columns.get_loc("SensorOcc000")
        X=dy.iloc[:,x1:x1+10]
        dy['average_occupancy']= X.mean(axis=1)
        dy.to_parquet(filepath)